##INSAID Capstone project - group 1015

###Installing and importing mysql.connector/python

In [152]:
!pip install --user mysql-connector-python

In [0]:
import mysql.connector
import pandas as pd

###Creating db connection and storing in capdb

In [0]:
 capdbConfig = {
     'host' : 'cpanel.insaid.co',
     'user' : 'student',
      'password' : 'student', 
      'database' : 'Capstone1',
      #increasing connection_timeout to support fetching larger results - doesn't seem to have any impact though
      'connection_timeout' : 1000,
      'use_pure' : False
      #use_pure = False ensures the C extension is used for MySQL connection which is faster than pure python connection
      #false is the default value for a connection in newer MySQL versions(>8); here it is included as version is not known
 }
 capdb = mysql.connector.connect(**capdbConfig)

###Creating python cursor for executing mysql queries, checking tables, structures and count

In [0]:
capdbCursor = capdb.cursor()
#below code intended to set max served packet size as 1 GB (to fetch large data) - but access denied in python client 
#capdbCursor.execute("SET GLOBAL max_allowed_packet=1024*1024*1024")

In [156]:
#closing and recreating cursor each time to ensure reduced load on next data fetch
capdbCursor.close()
capdbCursor = capdb.cursor()
capdbCursor.execute("show tables")
capdbTables = capdbCursor.fetchall()
for table in capdbTables:
    print(table)

('events_data',)
('gender_age_train',)
('phone_brand_device_model',)


In [157]:
capdbCursor.close()
capdbCursor = capdb.cursor()
capdbCursor.execute("describe events_data")
#fetchall fetches all records retrieved in cursor in a single go
#fetchall works fine when queried table is of small size
print(capdbCursor.fetchall())

[('event_id', 'bigint(20)', 'NO', '', None, ''), ('device_id', 'bigint(255)', 'YES', '', None, ''), ('timestamp', 'datetime', 'NO', '', None, ''), ('longitude', 'float(8,6)', 'YES', '', None, ''), ('latitude', 'float(8,6)', 'YES', '', None, ''), ('city', 'varchar(50)', 'NO', '', None, ''), ('state', 'varchar(50)', 'NO', '', None, '')]


In [158]:
capdbCursor.close()
capdbCursor = capdb.cursor()
query =   """select count(*)
            from events_data
            where state = 'delhi'"""
capdbCursor.execute(query)
#fetchone fetches single record retrieved in cursor
#fetchone is useful when only one record result is expected
#SQL connection is lost if fetchall is used for large size table
capdbRecords = capdbCursor.fetchone()
for record in capdbRecords:
    print(record)

751733


In [0]:
capdbCursor.close()
capdbCursor = capdb.cursor()
query =   """select *
            from events_data
            where state = 'delhi'
            limit 1"""
capdbCursor.execute(query)
#fetching single record to create pd dataframe
capdbRecords = capdbCursor.fetchone()
eventsData = DataFrame(capdbRecords)
eventsData.columns = capdbRecords.column_names()
#dropping the record as it will be fetched with all others in next step
#not doing this would create a duplicate record
eventsData.iloc[0:0]
print(eventsData)

In [0]:
capdbCursor.close()
capdbCursor = capdb.cursor()
query =   """select *
            from events_data
            where state = 'delhi'"""
capdbCursor.execute(query)
#fetching multiple records at a time and appending to previously created df
capdbRecords = capdbCursor.fetchmany(5)
for record in capdbRecords:
    dfTemp = DataFrame(capdbRecords)
    eventsData.append(dfTemp)
len(eventsData)

In [159]:
capdbCursor.close()
capdbCursor = capdb.cursor()
capdbCursor.execute("describe phone_brand_device_model")
print(capdbCursor.fetchall())

[('device_id', 'bigint(255)', 'NO', '', None, ''), ('phone_brand', 'varchar(2555)', 'NO', '', None, ''), ('device_model', 'varchar(2555)', 'NO', '', None, '')]


In [160]:
capdbCursor.close()
capdbCursor = capdb.cursor()
query =   """select count(*)
            from phone_brand_device_model m
            left join events_data e
            on m.device_id = e.device_id
            where e.state = 'delhi'"""
capdbCursor.execute(query)
capdbRecords = capdbCursor.fetchone()
for record in capdbRecords:
    print(record)

OperationalError: ignored

### Closing db connection

In [0]:
capdb.close()